<a href="https://colab.research.google.com/github/mikofarin12/practice_assignment/blob/master/Copy_of_188c_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#188C Homework 2: Twitter Data Analysis
In Homework 1, we collected Twitter data from 10 politicians(5 democrats and 5 republicans). In this homework, we will find the word useage frequency, which can reflect the focus of the two parties.

Any changes won't be saved on this example notebook. Please edit on your own copy. 

Please submit the pdf version of your notebook(on the top left menu, click File --> Print to get the pdf).

##Step 1: Load Data to notebook 

In [ ]:
# Import needed libraries
from google.colab import drive  # to mount Google Drive to Colab notebook
import tweepy                   # Python wrapper around Twitter API
import json
import pandas as pd
import csv
from datetime import date
from datetime import datetime
import time
import matplotlib.pyplot as plt
import os

#use the same code from hw1

# Connect to Twitter API using the secrets
auth = tweepy.OAuthHandler("aETIEZtuiP2gQV8wx2AxNLIEf", "CYywIpkSA6jLtoEjCG34If8weD4GxjoNBBi9aC4VumjDNzGSdc")
auth.set_access_token("1347299031593013248-2A0tKaIaM8x7sxiVt00Lc97MqKEmJZ", "PBmZnGZGAgVuS6rdeLcGcyU6tYITrcMpz2NUrQhBFfm6g")
api = tweepy.API(auth)

def get_all_tweets(screen_name):
  '''
  collect the lastest 1000 tweets from one specific user 
  input: twitter user name(str)
  '''

	# initialize a list to hold all the tweepy Tweets
  alltweets = []

	# make initial request for most recent tweets (use 200 as the maximum allowed count)
  recent_tweets = api.user_timeline(screen_name = screen_name,count=200)

	# save most recent tweets
  alltweets.extend(recent_tweets)
	
	# save the id of the oldest tweet 
  most_recent = alltweets[-1].id
	
	# write a loop to keep grabbing tweets until there are no tweets 
  #left to grab or reach 1000 tweets
  while len(recent_tweets) > 0:
    
    #collect tweets by 200 per, use max_id to set where to start and avoid duplicates
    recent_tweets = api.user_timeline(screen_name = screen_name,count=200, 
                                      max_id=most_recent)
    #save most recent tweets
    alltweets.extend(recent_tweets)
    
    #iterate to the next oldest tweet
    most_recent = alltweets[-1].id -1
    
    #if close to 1000, since we are grabbing tweets by 200 per request, 
    #stop the loop for a maximum of 1000
    if len(alltweets) >= 950:
      print("finished scraping", len(alltweets) ,"tweets from", screen_name, "account")
      break

  alltweets = [tweet.text for tweet in alltweets]
  return(alltweets)


# load only user tweets from Demnocrat party and save in the tweetsD list
tweetsD = get_all_tweets('JoeBiden')+ get_all_tweets('KamalaHarris') + get_all_tweets('BarackObama')+ get_all_tweets('GavinNewsom')+ get_all_tweets('NYGovCuomo')

# load only user tweets from Republican party and save in the tweetsR list
tweetsR = get_all_tweets('Mike_Pence')+ get_all_tweets('mikepompeo') + get_all_tweets('federalreserve')+ get_all_tweets('MittRomney')+ get_all_tweets('Schwarzenegger')


#checking if tweets collected correctly
tweetsD[:5]

finished scraping 1000 tweets from BarackObama account
finished scraping 1000 tweets from KamalaHarris account
finished scraping 1000 tweets from BarackObama account
finished scraping 1000 tweets from GavinNewsom account
finished scraping 1000 tweets from NYGovCuomo account
finished scraping 976 tweets from Mike_Pence account
finished scraping 1000 tweets from federalreserve account
finished scraping 999 tweets from MittRomney account
finished scraping 999 tweets from Schwarzenegger account


["In each of us, there's a story to be shared—a story that’s sacred and can bring us together. I had such a good time… https://t.co/dZ6zavY9Mr",
 'Even as we focus on fighting COVID-19, it’s important to recognize that there’s another pandemic raging right now—o… https://t.co/lBFmTp9nsT',
 'Sending my prayers to @TigerWoods and his family tonight—here’s to a speedy recovery for the GOAT of golf. If we’ve… https://t.co/fBtAC10hcv',
 'Last year, I sat down with my good friend Bruce @Springsteen for a long and meaningful conversation that touched on… https://t.co/m9hWwkrtDl',
 'For decades, John Lewis not only gave all of himself to the cause of freedom and justice, but inspired generations… https://t.co/trY8SGE47c']

##Step 2: Clean data
In the end, we want to calculate the word usage frequency. Before we do the actual word counting, we can see from the raw tweet data that the tweets include urls which are not supposed to be considered in the vocabulary. Therefore, first we need to remove urls from tweets.

Please the function remove_url(txt) below to remove urls from raw tweets and save new tweets in tweetsD_nourl and tweetsR_nourl

In [ ]:
import re
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

#taking url off tweets for Democrats
tweetsD_nourl= [remove_url(tweet) for tweet in tweetsD]

#taking url off tweets for Republicans
tweetsR_nourl =[remove_url(tweet) for tweet in tweetsR]

#checking if function is doing its job
tweetsD_nourl[:3]


['In each of us theres a story to be shareda story thats sacred and can bring us together I had such a good time',
 'Even as we focus on fighting COVID19 its important to recognize that theres another pandemic raging right nowo',
 'Sending my prayers to TigerWoods and his family tonightheres to a speedy recovery for the GOAT of golf If weve']

In [ ]:
# example usage
tweet = 'The weather is good. https://t.co/JLY1DntIlR'
remove_url(tweet)

'The weather is good'

Besides urls, we can see words can be in the form of upper case and lower case. For example, CAT and cat are actually the same word. Therefore, we want to transform all words into lower case to aviod repeat count.

Please go through each word in each tweets and use word.lower() to transform words into lower case form.

In [ ]:
# example usage
tweet = 'The weather is good. https://t.co/JLY1DntIlR'
tweet_nourl = remove_url(tweet)
tweet_lower = [word.lower() for word in tweet_nourl.split()]
print(tweet_lower)

tweet_lowerD = [tweet.lower().split() for tweet in tweetsD_nourl]


tweet_lowerR = [tweet.lower().split() for tweet in tweetsR_nourl]

#check if it works
tweet_lowerD[0]

['the', 'weather', 'is', 'good']


['in',
 'each',
 'of',
 'us',
 'theres',
 'a',
 'story',
 'to',
 'be',
 'shareda',
 'story',
 'thats',
 'sacred',
 'and',
 'can',
 'bring',
 'us',
 'together',
 'i',
 'had',
 'such',
 'a',
 'good',
 'time']

The last step before we do the actual counting is to remove stop words. There are words that do not add meaningful information to the text. These words referred to as “stop words” and include commonly appearing words such as who, what, you, etc.

Please use the nltk library to remove stop words in tweets.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# View a few words from the set
list(stop_words)[0:10]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['a', 'for', "you're", 'then', 'ain', 'doing', 't', 'myself', 'he', 'its']

In [ ]:
# example usage
tweet = ['the', 'weather', 'is', 'good']
tweet_nostop = [word for word in tweet if word not in stop_words]
print(tweet_nostop)

#testing code
tweet = ['last','year','i','sat','down','with','my','good','friend',
 'bruce','springsteen','for','a','long','and','meaningful','conversation',
 'that','touched','on']

tweet_nostop = [word for word in tweet if word not in stop_words]
print(tweet_nostop)

#cleaned tweets for Democrats
tweetD_nostop= [[word for word in tweet_words if word not in stop_words]
 for tweet_words in tweet_lowerD]

#checking if code worked
tweetD_nostop[10]

#cleaned tweets for Republicans
tweetR_nostop= [[word for word in tweet_words if word not in stop_words]
 for tweet_words in tweet_lowerR]



['weather', 'good']
['last', 'year', 'sat', 'good', 'friend', 'bruce', 'springsteen', 'long', 'meaningful', 'conversation', 'touched']


##Step 3: build vocabulary
After we get tweet_nostop, we can build vocabulary and count frequency useage of these two parties.

1. Please find the top 30 most used word lists of these two parties as TopD and TopR. 

You may find collections.Counter() useful.

In [ ]:
# print TopD and TopR and their corresponding frequency 
#import these libraries to perform the count
import itertools
import collections

#clean our vector and create a list of words

#democreats count
cleaned_tweetsD = list(itertools.chain(*tweetD_nostop))

# use suggested word count 
TopD = collections.Counter(cleaned_tweetsD)
TopD= TopD.most_common(30)

#Republican count
cleaned_tweetsR = list(itertools.chain(*tweetR_nostop))

# use suggested word count 
TopR = collections.Counter(cleaned_tweetsR)
TopR = TopR.most_common(30)


print(TopD)
print(TopR)


[('rt', 1043), ('today', 396), ('joebiden', 391), ('new', 370), ('vote', 338), ('people', 268), ('us', 266), ('covid', 260), ('day', 252), ('one', 230), ('update', 227), ('make', 226), ('get', 225), ('covid19', 219), ('leaders', 187), ('president', 187), ('yesterday', 186), ('todays', 176), ('first', 174), ('country', 170), ('health', 169), ('time', 161), ('tests', 157), ('americans', 156), ('need', 153), ('reported', 152), ('im', 146), ('election', 145), ('amp', 144), ('live', 143)]
[('rt', 907), ('president', 357), ('amp', 295), ('today', 290), ('us', 258), ('schwarzenegger', 244), ('thank', 212), ('great', 205), ('america', 195), ('realdonaldtrump', 188), ('federalreserve', 186), ('years', 177), ('feddata', 177), ('weekly', 175), ('federal', 167), ('see', 161), ('live', 155), ('day', 154), ('chair', 141), ('back', 140), ('get', 136), ('reserve', 133), ('vice', 122), ('new', 122), ('watch', 119), ('american', 118), ('balancesheet', 117), ('vote', 115), ('people', 114), ('mikepence', 

2. After finding TopD and TopR, please find the intersection of TopD and TopR as the vocabulary. Use two arrays to represent the word frequency of the two parties using words in the vocabulary. 

In [ ]:
# example array
vocabulary = ['weather', 'good']
freqD = [100, 30] # use weather 100 times and good 30 times
freqR = [50, 100] # use weather 50 times and good 100 times


#collapse the list of words to find intersection
collapsed_listD = list(itertools.chain(*TopD))
collapsed_listR = list(itertools.chain(*TopR))

#use code from online to define a function for finding intersection
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

#checking the intersection of the top vocabulary
print(intersection(collapsed_listD,collapsed_listR))
intersection_vocab = (intersection(collapsed_listD,collapsed_listR))

#just getting the intersection of words, not numbers
intersection_vocab = intersection_vocab[0:9] + intersection_vocab[11:13]
print(intersection_vocab)

#now we take out the non intersection words from out list Democracts
tweetD_nostop_intersection =[[word for word in tweet_words if word in intersection_vocab]
 for tweet_words in tweetD_nostop]

#democreats count intersection
cleaned_tweetsD_intersection = list(itertools.chain(*tweetD_nostop_intersection))
intersection_words_D = collections.Counter(cleaned_tweetsD_intersection)

print(intersection_words_D)

#now we take out the non intersection words from out list Democracts
tweetR_nostop_intersection =[[word for word in tweet_words if word in intersection_vocab]
 for tweet_words in tweetR_nostop]


#Republican count intersection
cleaned_tweetsR_intersection = list(itertools.chain(*tweetR_nostop_intersection))
intersection_words_R = collections.Counter(cleaned_tweetsR_intersection)


print(intersection_words_R)

['rt', 'today', 'new', 'vote', 'people', 'us', 'day', 'get', 'president', 186, 161, 'amp', 'live']
['rt', 'today', 'new', 'vote', 'people', 'us', 'day', 'get', 'president', 'amp', 'live']
Counter({'rt': 1043, 'today': 396, 'new': 370, 'vote': 338, 'people': 268, 'us': 266, 'day': 252, 'get': 225, 'president': 187, 'amp': 144, 'live': 143})
Counter({'rt': 907, 'president': 357, 'amp': 295, 'today': 290, 'us': 258, 'live': 155, 'day': 154, 'get': 136, 'new': 122, 'vote': 115, 'people': 114})


3. Here we will use chi-square test to find out whether parties have significant word useage difference.

Please compute the p value using chi-square test library https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html and use p<0.05 as the accepted significance level

More about chi-square test can found here https://math.hws.edu/javamath/ryan/ChiSquare.html.

In [ ]:
#now we create two arrays for word frequncy
#Democrat Counter({'rt': 1043, 'today': 396, 'new': 370, 'vote': 338, 'people': 268, 'us': 266, 'day': 252, 'get': 225, 'president': 187, 'amp': 144, 'live': 143})
freqD_intersection  = [1043,396,370,338,268,266,252,225,187,144,143]

#Republican Counter({'rt': 907,'today': 290 , 'new': 122, 'vote': 115','people': 114, 'us': 258, 'day': 154, 'get': 136, president': 357, 'amp': 295, 'live': 155, })
freqR_intersection = [907,290,122,115,114,258,154,136,357,295,155]

from scipy.stats import chisquare
chisquare(freqD_intersection ,freqR_intersection)

Power_divergenceResult(statistic=1483.7569366314883, pvalue=0.0)

## Since, p-value is 0.0 meaning <0.05 we reject the null hypothesis that there's no difference between the means and conclude that a significant difference does exist. In, context we find the the languages between democrats and republicans on twitter differ.